In [3]:
# 1：ライブラリのインポート--------------------------------
import numpy as np #numpyという行列などを扱うライブラリを利用
import pandas as pd #pandasというデータ分析ライブラリを利用
import matplotlib.pyplot as plt #プロット用のライブラリを利用
from sklearn import svm, metrics, preprocessing #機械学習用のライブラリを利用
from sklearn.model_selection import cross_validate, cross_val_score, train_test_split
from mlxtend.plotting import plot_decision_regions #学習結果をプロットする外部ライブラリを利用
# from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from preprocessing import preprocess_dataset
from utils import load_dataset


import preprocessing

from importlib import reload
reload(preprocessing)

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

import pickle

from remove_stop_words import download_stopwords,create_stopwords

In [4]:



# 2：Wineのデータセットを読み込む--------------------------------
# df_wine_all=pd.read_csv('data/amazon_reviews_multilingual_JP_v1_00.tsv',delimiter='\t',header=None)
# df_wine_all=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)
#品種(0列、1～3)と色（10列）とプロリンの量(13列)を使用する
# df_wine=df_wine_all[[7,12]]
# df_wine.columns = [u'star_rating', u'review_body']
# pd.DataFrame(df_wine)  #この行を実行するとデータが見れる


# 3：プロットしてみる------------------------------------------------------
# %matplotlib inline
# x=df_wine["color"]
# y=df_wine["proline"]
# z=df_wine["star_rating"]
#plt.scatter(x,y, c=z)
#plt.show

# 4：データの整形-------------------------------------------------------

x,y=load_dataset('data/amazon_reviews_multilingual_JP_v1_00_final.tsv')	
x=preprocessing.preprocess_dataset(x)

from janome.tokenizer import Tokenizer
t=Tokenizer(wakati=True)

path = "stop_words.txt"
stop_words = create_stopwords(path)
print(stop_words)

vectorizer=TfidfVectorizer(tokenizer=t.tokenize,stop_words=stop_words)
tfidf=vectorizer.fit_transform(x)

vocab=vectorizer.get_feature_names()
x=pd.DataFrame(tfidf.toarray(),columns=vocab).to_numpy()
print(x.shape)

                                             review_body  star_rating
0           残念ながら…残念ながら…趣味ではありませんでした。ケルト音楽の範疇にも幅があるのですね…            1
1                 残念ながら…残念ながら…趣味ではありませんでした。正直退屈…眠気も起きない…            1
2         ドリームキャスト。素晴らしいパフォーマンス。ミュージカル映画版の物足りない歌唱とは違います。            5
3      やっぱりマスト。専門的な事を言わずにお勧めレコメを書きたいのですが、文才が無いので無理でした...            5
4      コスパ的には十分。今までの環境（Core2 Duo E4600)に比べれば十分に快適になりま...            4
...                                                  ...          ...
24995  サイバーニンジャ 3Dガンシューティング。操作は照準、防御、手裏剣のみですが、システムが破綻...            2
24996  宇宙戦艦リアルタイムシミュレーション。宇宙戦艦の乗組員を作り、艦内の設備を補強、メンテナンス...            2
24997  電子ボードゲーム。ルールがちょっと特殊な陣取りゲーム<br />コマをタップすると上下左右１...            2
24998  変則型さめがめ。ルールはさめがめに似ています<br />タップするとブロックが落ちるTHIN...            2
24999  空白の14年を補完する物語をつくってください。シンジがいなくともいい。まず、シンジが助けたと...            2

[25000 rows x 2 columns]
['あそこ', 'あたり', 'あちら', 'あっち', 'あと', 'あな', 'あなた', 'あれ', 'いくつ', 'いつ', 'いま', 'いや', 'いろいろ', 'うち', 'おおまか', 'おまえ', 'おれ', 'がい', 'かく', 'かたち', '

In [13]:
filename='models/Amazon_LR.sav'

#解説 5：機械学習で分類する---------------------------------------------------
# lr=LinearSVC()#loss='squared_hinge' #loss="hinge", loss="log"

# # 6：K分割交差検証（cross validation）で性能を評価する---------------------
# scores=cross_val_score(lr, x, y, cv=10)
# print("平均正解率 = ", scores.mean())
# print("正解率の標準偏差 = ", scores.std())

# 7：トレーニングデータとテストデータに分けて実行してみる------------------
X_train, X_test, Y_train, Y_test=train_test_split(x,y, test_size=0.2, random_state=0)
lr.fit(X_train, Y_train)

pickle.dump(lr, open(filename, 'wb'))

lr = pickle.load(open(filename, 'rb'))

print(lr.coef_)

#正答率を求める
Y_pred=lr.predict(X_test)
X_pred=lr.predict(X_train)

print('train:')
print('confusion matrix = \n', confusion_matrix(y_true=Y_train, y_pred=X_pred))
print('accuracy = ', accuracy_score(y_true=Y_train, y_pred=X_pred))
print('precision = ', precision_score(y_true=Y_train, y_pred=X_pred,average='binary'))
print('recall = ', recall_score(y_true=Y_train, y_pred=X_pred,average='binary'))
print('f1 score = ', f1_score(y_true=Y_train, y_pred=X_pred,average='binary'))

print()

print('test:')
print('confusion matrix = \n', confusion_matrix(y_true=Y_test, y_pred=Y_pred))
print('accuracy = ', accuracy_score(y_true=Y_test, y_pred=Y_pred))
print('precision = ', precision_score(y_true=Y_test, y_pred=Y_pred,average='binary'))
print('recall = ', recall_score(y_true=Y_test, y_pred=Y_pred,average='binary'))
print('f1 score = ', f1_score(y_true=Y_test, y_pred=Y_pred,average='binary'))

print()

# Twitter analysis
new=pd.read_csv('data/Lesson_analysis.tsv',sep='\t')
new=new[new.sentiment!=0]
new.tweet=preprocessing.preprocess_dataset(new.tweet)
text=new.tweet

text=vectorizer.transform(text)
text=pd.DataFrame(text.toarray(),columns=vocab).to_numpy()
print(text.shape)

Twitter_pred=lr.predict(text)
# Twitter_prob=lr.predict_proba(text)

Twitter_right=new.sentiment

print(type(Twitter_pred))
print(type(Twitter_right))
print(Twitter_pred)
print(Twitter_right.to_numpy())
Twitter_right=Twitter_right.to_numpy()

wrong_sum=0

for index,sentence in enumerate(Twitter_pred):
    if Twitter_right[index]!=Twitter_pred[index]:
        temp_dict={}
        print(index+1)
        print(new.tweet.to_numpy()[index])
        print('right=',Twitter_right[index],'wrong=',Twitter_pred[index])
        # print(Twitter_prob[index])
        for vector_number, value in enumerate(text[index]):
            if value != 0:
                temp_dict[vocab[vector_number]]=value*lr.coef_[0][vector_number]
        temp_dict = sorted(temp_dict.items(), key=lambda x:x[1])
        print(temp_dict)
        print()
        wrong_sum+=1

print(wrong_sum)

print('Twitter:')
print('confusion matrix = \n', confusion_matrix(y_true=Twitter_right, y_pred=Twitter_pred))
print('accuracy = ', accuracy_score(y_true=Twitter_right, y_pred=Twitter_pred))
print('precision = ', precision_score(y_true=Twitter_right, y_pred=Twitter_pred,average='binary'))
print('recall = ', recall_score(y_true=Twitter_right, y_pred=Twitter_pred,average='binary'))
print('f1 score = ', f1_score(y_true=Twitter_right, y_pred=Twitter_pred,average='binary'))







#plotする
# X_train_plot=np.vstack(X_train)
# train_label_plot=np.hstack(train_label)
# X_test_plot=np.vstack(X_test)
# test_label_plot=np.hstack(test_label)
#plot_decision_regions(X_train_plot, train_label_plot, clf=clf_result, res=0.01) #学習データをプロット
# plot_decision_regions(X_test_plot, test_label_plot, clf=clf_result, res=0.01, legend=2) #テストデータをプロット

# 8：任意のデータに対する識別結果を見てみる------------------
#predicted_label=clf_result.predict([1,-1])
#print("このテストデータのラベル = ", predicted_label)
 
# 9：識別平面の式を手に入れる--------------------------------
# print(clf_result.intercept_)

# print(clf_result.coef_ )  #coef[0]*x+coef[1]*y+intercept=0

[[ 0.06141484 -0.11828892 -0.12855383 ... -0.23853273  0.11398188
   0.25805045]]
train:
confusion matrix = 
 [[3980   12]
 [  17 3991]]
accuracy =  0.996375
precision =  0.9970022483137647
recall =  0.9957584830339321
f1 score =  0.996379977530895

test:
confusion matrix = 
 [[932  76]
 [ 88 904]]
accuracy =  0.918
precision =  0.9224489795918367
recall =  0.9112903225806451
f1 score =  0.9168356997971602

(356, 34191)


AttributeError: 'LinearSVC' object has no attribute 'predict_proba'

In [21]:
lr.coef_[0][np.argsort(lr.coef_)[0,::-1]]

array([ 5.21559478,  4.88120161,  4.80823781, ..., -4.93342719,
       -5.47803294, -7.13136263])

In [22]:
vocab_np = np.array(vocab)

In [23]:
token_w = [[word,weight] for word,weight in zip(vocab_np[np.argsort(lr.coef_)[0,::-1]],lr.coef_[0][np.argsort(lr.coef_)[0,::-1]])]

In [34]:
for word,weight in token_w[:100:-1]:
    print(word+' : '+str(weight) )

ない : -7.131362629363584
残念 : -5.478032939634874
ん : -4.933427185791213
し : -4.474012333655824
ませ : -4.467002526590788
最悪 : -4.014353482172309
? : -3.8761720376779576
.. : -3.6183891763691256
のに : -3.588175000671565
だけ : -3.1652161801624796
返品 : -3.0607941936304015
駄作 : -3.01533448632665
評価 : -2.9723055370674647
か : -2.928980926047605
悪い : -2.8887017701738102
つまらない : -2.829577479643984
すぎ : -2.6802821240836567
吹き替え : -2.6451417961056474
が : -2.5996055236666358
全く : -2.5490808846941087
ダメ : -2.483328296785869
がっかり : -2.453353376221637
最低 : -2.4493095311422466
声優 : -2.1789898091492774
こんな : -2.146517781224863
退屈 : -2.0855452275120783
ひどい : -2.0329072021372636
なかっ : -1.9390798260316666
酷い : -1.9235280214711645
途中 : -1.921482359108353
..。 : -1.910650868337289
なら : -1.852973680103215
ガッカリ : -1.8328181159256633
再生 : -1.8217113024839708
てる : -1.7952135641268843
しか : -1.7951387882188081
無駄 : -1.766528128338444
インストール : -1.7261840260245174
期待 : -1.7035418402960603
失敗 : -1.6997925295002168
無い : -